In [1]:
import torch
from torch import optim, nn
import numpy as np
from torchvision import transforms as T
from tqdm import tqdm
import os
import argparse
import yaml
import json
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.signal import convolve2d
import os
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
from dataset import MotionPSF, DefocusPSF, GaussianPSF, AnonymizePSF, CreatePatchDataset, load_image, generate_blurred_image

/home/msouda/.pyenv/versions/3.9.18/envs/face_detection/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#A CHANGER, PROPRE A CHAQUE DATASET DE BASE
dataset_root = '/home/msouda/Datasets'
dataset_name = 'BSDS300/images'
target_name = 'BSDS300_blurred'
dataset_path = os.path.join(dataset_root, dataset_name)
target_path = os.path.join(dataset_root, target_name)
if not os.path.exists(target_path):
    os.mkdir(target_path)
classes = os.listdir(dataset_path)
images = []
target_images = []
for c in tqdm(classes):
    if len(c.split('.')) > 1:
        continue
    tmp_list = os.listdir(os.path.join(dataset_path, c))
    images.extend([os.path.join(dataset_path, c, f) for f in tmp_list])
    target_images.extend([os.path.join(target_path, c+'_'+f) for f in tmp_list])

print(images)

100%|██████████| 3/3 [00:00<00:00, 1850.70it/s]

['/home/msouda/Datasets/BSDS300/images/test/143090.jpg', '/home/msouda/Datasets/BSDS300/images/test/54082.jpg', '/home/msouda/Datasets/BSDS300/images/test/12084.jpg', '/home/msouda/Datasets/BSDS300/images/test/126007.jpg', '/home/msouda/Datasets/BSDS300/images/test/175032.jpg', '/home/msouda/Datasets/BSDS300/images/test/304074.jpg', '/home/msouda/Datasets/BSDS300/images/test/85048.jpg', '/home/msouda/Datasets/BSDS300/images/test/271035.jpg', '/home/msouda/Datasets/BSDS300/images/test/167062.jpg', '/home/msouda/Datasets/BSDS300/images/test/182053.jpg', '/home/msouda/Datasets/BSDS300/images/test/41033.jpg', '/home/msouda/Datasets/BSDS300/images/test/42012.jpg', '/home/msouda/Datasets/BSDS300/images/test/43074.jpg', '/home/msouda/Datasets/BSDS300/images/test/14037.jpg', '/home/msouda/Datasets/BSDS300/images/test/69015.jpg', '/home/msouda/Datasets/BSDS300/images/test/109053.jpg', '/home/msouda/Datasets/BSDS300/images/test/108082.jpg', '/home/msouda/Datasets/BSDS300/images/test/236037.jpg',

In [3]:
#RANDOM choice of blur type
blur_dict = {'clear': 0, 'motion': 1, 'defocus': 2, 'gaussian': 3, 'anonym':4}
PSF_dict = {'clear': None, 'motion': MotionPSF, 'defocus': DefocusPSF, 'gaussian': GaussianPSF, 'anonym': AnonymizePSF}
annotations = []
for i in tqdm(range(len(images))):
    blur_type = np.random.choice(list(blur_dict.keys()))
    if not os.path.exists(images[i]):
        print('Image not found :', images[i])
        continue
    if blur_type == 'clear':
        cv2.imwrite(target_images[i], cv2.cvtColor(load_image(images[i]), cv2.COLOR_RGB2BGR))
    else:
        PSF = PSF_dict[blur_type]()
        
        blurred_image = generate_blurred_image(images[i], PSF)
        if not target_images[i].endswith('.jpg'):
            target_images[i] = target_images[i]+'.jpg'
        cv2.imwrite(target_images[i], cv2.cvtColor(blurred_image, cv2.COLOR_RGB2BGR))
    annotations.append((images[i], target_images[i], blur_dict[blur_type]))

df = pd.DataFrame(annotations, columns=['image', 'blurred_image', 'blur_type'])
path_to_df = os.path.join(dataset_path, 'BSDS300_blurred.csv')
df[['blurred_image', 'blur_type']].to_csv(path_to_df, index=False, header=False)

100%|██████████| 300/300 [01:01<00:00,  4.89it/s]


In [4]:
transform = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomVerticalFlip(),
    T.ToTensor(),
])


train_dataset = CreatePatchDataset(annotations_file=path_to_df, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=False, num_workers = 4)

patch_target = '/home/msouda/Datasets/SFA_patches_300/'
if not os.path.exists(patch_target):
    os.mkdir(patch_target)


for img, path, label in tqdm(train_dataloader):
    new_path = patch_target + path[0].split('/')[-1]

    Image.fromarray((img[0].permute(1,2,0).numpy()*255).astype(np.uint8)).save(new_path)

path = os.listdir(patch_target)
o_path = pd.Series(path).apply(lambda x: '_'.join(x.split('_')[:-2])+'.jpg')
ann = df[['blurred_image','blur_type']].rename(columns = {'blurred_image': 'img_path', 'blur_type': 'label'}).assign(img_path= lambda x: x['img_path'].apply(lambda y: y.split('/')[-1]))
p_ann = pd.DataFrame({'img_path': o_path, 'p_path': path})

annot = pd.merge(ann, p_ann, on='img_path').drop('img_path', axis=1)[['p_path', 'label']].assign(p_path=lambda x: patch_target + x.p_path)
annot.to_csv(os.path.join(patch_target, 'annotations.csv'), header=None, index=None)

7200


100%|██████████| 7200/7200 [00:04<00:00, 1651.27it/s]
